# Парсер новостных текстов с сайтов РБК и Лента.ру

## Импорт библиотек и описание классов

In [ ]:
# Установка библиотек
!pip install bs4
!pip install openpyxl

In [ ]:
# Импорт библиотек
import requests as rq
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from IPython import display

Ниже описаны классы для извлечения данных.

Используются поисковые движки сайтов, возвращающие по запросу json таблицы с информацией о статьях.

При их использовании есть различные ограничения, например, на количество статей в запросе. Так на сайте РБК выдается максимум 100 статей, в то время как на Ленте.ру можно получить сразу 1000. По моим наблюдениям, запросы на Лента.ру ограничены временем, и соответственно, точное ограничение найти не получится. Более того, на Лента.ру сразу выдается текст статьи в json таблице, с РБК приходится дополнительно парсить текст со страниц.  

### Парсер для Лента.ру

In [ ]:
class lentaRu_parser:
    def __init__(self):
        pass

    def _get_url(self, param_dict: dict) -> str:
        """
        Возвращает URL для запроса json таблицы со статьями

        url = 'https://lenta.ru/search/v2/process?'\
        + 'from=0&'\                       # Смещение
        + 'size=1000&'\                    # Кол-во статей
        + 'sort=2&'\                       # Сортировка по дате (2), по релевантности (1)
        + 'title_only=0&'\                 # Точная фраза в заголовке
        + 'domain=1&'\                     # ??
        + 'modified%2Cformat=yyyy-MM-dd&'\ # Формат даты
        + 'type=1&'\                       # Материалы. Все материалы (0). Новость (1)
        + 'bloc=4&'\                       # Рубрика. Экономика (4). Все рубрики (0)
        + 'modified%2Cfrom=2020-01-01&'\
        + 'modified%2Cto=2020-11-01&'\
        + 'query='                         # Поисковой запрос
        """
        hasType = int(param_dict["type"]) != 0
        hasBloc = int(param_dict["bloc"]) != 0

        url = (
            "https://lenta.ru/search/v2/process?"
            + "from={}&".format(param_dict["from"])
            + "size={}&".format(param_dict["size"])
            + "sort={}&".format(param_dict["sort"])
            + "title_only={}&".format(param_dict["title_only"])
            + "domain={}&".format(param_dict["domain"])
            + "modified%2Cformat=yyyy-MM-dd&"
            + "type={}&".format(param_dict["type"]) * hasType
            + "bloc={}&".format(param_dict["bloc"]) * hasBloc
            + "modified%2Cfrom={}&".format(param_dict["dateFrom"])
            + "modified%2Cto={}&".format(param_dict["dateTo"])
            + "query={}".format(param_dict["query"])
        )

        return url

    def _get_search_table(self, param_dict: dict) -> pd.DataFrame:
        """
        Возвращает pd.DataFrame со списком статей
        """
        url = self._get_url(param_dict)
        r = rq.get(url)
        search_table = pd.DataFrame(r.json()["matches"])

        return search_table

    def get_articles(
        self, param_dict, time_step=37, save_every=5, save_excel=True
    ) -> pd.DataFrame:
        """
        Функция для скачивания статей интервалами через каждые time_step дней
        Делает сохранение таблицы через каждые save_every * time_step дней

        param_dict: dict
        ### Параметры запроса
        ###### project - раздел поиска, например, rbcnews
        ###### category - категория поиска, например, TopRbcRu_economics
        ###### dateFrom - с даты
        ###### dateTo - по дату
        ###### offset - смещение поисковой выдачи
        ###### limit - лимит статей, максимум 100
        ###### query - поисковой запрос (ключевое слово), например, РБК

        """
        param_copy = param_dict.copy()
        time_step = timedelta(days=time_step)
        dateFrom = datetime.strptime(param_copy["dateFrom"], "%Y-%m-%d")
        dateTo = datetime.strptime(param_copy["dateTo"], "%Y-%m-%d")
        if dateFrom > dateTo:
            raise ValueError("dateFrom should be less than dateTo")

        out = pd.DataFrame()
        save_counter = 0

        while dateFrom <= dateTo:
            param_copy["dateTo"] = (dateFrom + time_step).strftime("%Y-%m-%d")
            if dateFrom + time_step > dateTo:
                param_copy["dateTo"] = dateTo.strftime("%Y-%m-%d")
            print(
                "Parsing articles from "
                + param_copy["dateFrom"]
                + " to "
                + param_copy["dateTo"]
            )
            out = out.append(self._get_search_table(param_copy), ignore_index=True)
            dateFrom += time_step + timedelta(days=1)
            param_copy["dateFrom"] = dateFrom.strftime("%Y-%m-%d")
            save_counter += 1
            if save_counter == save_every:
                display.clear_output(wait=True)
                out.to_excel("/tmp/checkpoint_table.xlsx")
                print("Checkpoint saved!")
                save_counter = 0

        if save_excel:
            out.to_excel(
                "lenta_{}_{}.xlsx".format(param_dict["dateFrom"], param_dict["dateTo"])
            )
        print("Finish")

        return out

## Пример выгрузки данных

### Лента.ру

* __query__ - поисковой запрос (ключевое слово)

* __offset__ - cмещение поисковой выдачи (от 0 до __size__)

* __size__ - количество статей. Ограничено время запроса, точного лимита нет. 1000 работает почти всегда

* __sort__ - сортировка по дате: (2) - по убыванию, (3) - по возрастанию; по релевантности (1)

* __title_only__ - точная фраза в заголовке (1)

* __domain__ - ?

* __material__ - материалы: Все материалы (0). Новость (1). ["0", "1", "2", "3", "4", ...]

* __block__ - рубрика: Экономика (4). Все рубрики (0). ["0", "1", "2", "3", "4", ...]

* __dateFrom__ - с даты

* __dateTo__ - по дату

_Чтобы не специфировать параметр, оставляем поле пустым_

In [ ]:
# Задаем тут параметры
use_parser = "LentaRu"

query = ""
offset = 0
size = 1000
sort = "3"
title_only = "0"
domain = "1"
material = "0"
bloc = "0"
dateFrom = "2020-01-01"
dateTo = "2023-10-21"

if use_parser == "LentaRu":
    param_dict = {
        "query": query,
        "from": str(offset),
        "size": str(size),
        "dateFrom": dateFrom,
        "dateTo": dateTo,
        "sort": sort,
        "title_only": title_only,
        "type": material,
        "bloc": bloc,
        "domain": domain,
    }

print(use_parser, "- param_dict:", param_dict)

LentaRu - param_dict: {'query': '', 'from': '0', 'size': '1000', 'dateFrom': '2020-01-01', 'dateTo': '2023-10-21', 'sort': '3', 'title_only': '0', 'type': '0', 'bloc': '0', 'domain': '1'}


In [ ]:
# Тоже будем собирать итеративно, правда можно ставить time_step побольше, т.к.
# больше лимит на запрос статей. И Работает быстрее :)
assert use_parser == "LentaRu"
parser = lentaRu_parser()
tbl = parser.get_articles(
    param_dict=param_dict, time_step=37, save_every=5, save_excel=True
)
print(len(tbl.index))

Checkpoint saved!
Parsing articles from 2023-08-23 to 2023-09-29


<ipython-input-24-e7bc6880237a>:90: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out = out.append(self._get_search_table(param_copy), ignore_index=True)


Parsing articles from 2023-09-30 to 2023-10-21


<ipython-input-24-e7bc6880237a>:90: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out = out.append(self._get_search_table(param_copy), ignore_index=True)


Finish
37000


In [ ]:
print(f"shape tbl{tbl.shape}")
print(f"unique categories {tbl.bloc.unique()}")

shape tbl(37000, 16)
unique categories [ 1  6  4  5  2  7  8 37  3 47  9 48 12 49  0 53 86 87 40]


In [ ]:
tbl

,docid,url,title,modified,lastmodtime,type,domain,status,part,bloc,tags,image_url,pubdate,text,rightcol,snippet
0,943560,https://lenta.ru/news/2020/01/01/trudovaya/,В России ввели электронные трудовые книжки,1577836860,1577866854,1,1,0,0,1,[2],https://icdn.lenta.ru/images/2019/12/30/15/201...,1577836860,Фото: Юрий Мартьянов /« Коммерсантъ» В России...,Работодатели смогут передавать сведения в Пенс...,Фото: Юрий Мартьянов /« Коммерсантъ» В ... кн...
1,943835,https://lenta.ru/news/2020/01/01/zavorotnuk/,Заворотнюк поблагодарила поклонников за поддержку,1577838422,1577838422,1,1,0,0,6,[18],https://icdn.lenta.ru/images/2020/01/01/03/202...,1577838422,Фото: Анатолий Ломохов / Globallookpress.com П...,Соответствующая запись появилась в Instagram-а...,Фото: Анатолий Ломохов / Globallookpress.... г...
2,943747,https://lenta.ru/news/2020/01/01/org/,В России утвердили стандарты органической еды,1577838649,1577838649,1,1,0,0,4,"[7, 8]",https://icdn.lenta.ru/images/2019/12/31/13/201...,1577838649,Фото: Rupert Oberhäuser / Globallookpress.com ...,Вступил в силу принятый в 2018 году закон,Фото: Rupert Oberhäuser / ... для производител...
3,943836,https://lenta.ru/news/2020/01/01/sleep/,Объяснена опасность неправильного сна,1577839980,1577964161,1,1,0,0,5,[16],https://icdn.lenta.ru/images/2020/01/01/03/202...,1577839980,Фото: Vladislav Muslakov / Unsplash Спящие бол...,Спящие слишком мало или слишком много более по...,"Фото: Vladislav Muslakov / Unsplash ..., чем т..."
4,943634,https://lenta.ru/news/2020/01/01/theaters/,Российские театры освободили от налогов,1577840882,1577840883,1,1,0,0,4,[7],https://icdn.lenta.ru/images/2019/12/30/20/201...,1577840882,Фото: Константин Кокошкин / «Коммерсантъ» Росс...,Обнулить базу удастся при условии строго следо...,Фото: Константин Кокошкин / «Коммерсантъ»... п...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36995,1501182,https://lenta.ru/articles/2023/10/03/oceanrise/,Целые страны уходят под воду.,1696280645,1696280645,2,1,0,1,12,[281],https://icdn.lenta.ru/images/2023/09/29/14/202...,1696280645,Фото: Bram Janssen / AP Мария Грушко Из-за гло...,Все началось с крошечного государства в Тихом ...,Фото: Bram Janssen / AP Мария Грушко Из-... уг...
36996,1492071,https://lenta.ru/articles/2023/10/03/cocktails...,10 лучших коктейлей с ромом.,1696280650,1696280651,2,1,0,1,9,[44],https://icdn.lenta.ru/images/2023/09/12/14/202...,1696280650,Фото: Pexels Юлия Козинцева Коктейли на основе...,Как приготовить вкусные коктейли в домашних ус...,Фото: Pexels Юлия Козинцева Коктейли на ... ва...
36997,1492010,https://lenta.ru/extlink/2023/09/11/dohodniye_...,Доходные дома.,1696280726,1696280726,15,1,0,1,0,[],https://icdn.lenta.ru/images/2023/10/02/17/202...,1696280726,,Почему спустя много лет они стали снова появля...,
36998,1503340,https://lenta.ru/news/2023/10/03/peregovory/,Чехия назвала сроки переговоров о вступлении У...,1696281180,1696281420,1,1,0,0,2,[1],https://icdn.lenta.ru/images/2023/10/03/00/202...,1696281180,Ян Липавский Фото: Valentyn Ogirenko / Reuters...,Чехия назвала сроки переговоров о вступлении У...,Ян Липавский Фото: Valentyn Ogirenko / ... осе...


In [ ]:
tbl.bloc.unique()

array([ 1,  6,  4,  5,  2,  7,  8, 37,  3, 47,  9, 48, 12, 49,  0, 53, 86,
       87, 40])

In [ ]:
tbl.groupby("bloc").agg({"docid": "count"})

,docid
bloc,
0,272
1,7296
2,5536
3,3572
4,3531
5,1902
6,1375
7,1659
8,2637


In [ ]:
news_category = {1: "russia_news", 2: "world", 3: "former_ussr", 4: "economy", 5: 'military_structures', 6: 'наука и техника', 7: 'culture', 8:'спорт', 9: 'internet', 10: 'ценности',
                 11: 'путешествия', 12: 'из жизни', 13: 'enviroment', 14:'about self'}

In [ ]:
tbl.loc[tbl.bloc.isin([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14])].groupby("bloc").agg({"docid": "count"})

,docid
bloc,
1,7296
2,5536
3,3572
4,3531
5,1902
6,1375
7,1659
8,2637
9,1355


In [ ]:
df = tbl.loc[tbl.bloc.isin([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14])]

In [ ]:
df.head(2)

,docid,url,title,modified,lastmodtime,type,domain,status,part,bloc,tags,image_url,pubdate,text,rightcol,snippet
0,943560,https://lenta.ru/news/2020/01/01/trudovaya/,В России ввели электронные трудовые книжки,1577836860,1577866854,1,1,0,0,1,[2],https://icdn.lenta.ru/images/2019/12/30/15/201...,1577836860,Фото: Юрий Мартьянов /« Коммерсантъ» В России...,Работодатели смогут передавать сведения в Пенс...,Фото: Юрий Мартьянов /« Коммерсантъ» В ... кн...
1,943835,https://lenta.ru/news/2020/01/01/zavorotnuk/,Заворотнюк поблагодарила поклонников за поддержку,1577838422,1577838422,1,1,0,0,6,[18],https://icdn.lenta.ru/images/2020/01/01/03/202...,1577838422,Фото: Анатолий Ломохов / Globallookpress.com П...,Соответствующая запись появилась в Instagram-а...,Фото: Анатолий Ломохов / Globallookpress.... г...


In [ ]:
df.to_csv("lenta_news_big", index=False)

In [ ]:
df = pd.read_csv("lenta_news")

In [ ]:
df.columns

In [ ]:
# Еще раз зададим словарь с типами новостей
news_category = {1: "russia_news", 2: "world", 3: "former_ussr", 4: "economy"}

In [ ]:
# Дропиним столбцы, которые мы не будем использовать
df.drop(
    columns=[
        "docid",
        "modified",
        "lastmodtime",
        "type",
        "domain",
        "status",
        "part",
        "tags",
        "image_url",
        "rightcol",
    ],
    inplace=True,
)

# Подтянем категории новостей из словаря
df.bloc = df.bloc.map(news_category)

# Заменем названия колонок на более читаемое
df.rename(
    columns={"text": "content", "snippet": "subtitle", "bloc": "topic"}, inplace=True
)

# Переведем timestamp в стринг с датой и временем
df["pubdate"] = df["pubdate"].apply(lambda x: str(datetime.fromtimestamp(x)))

# Переименуем столбец содержащий дату и время
df.rename(columns={"pubdate": "datetime"}, inplace=True)

<ipython-input-38-249fd0218283>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(
<ipython-input-38-249fd0218283>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.bloc = df.bloc.map(news_category)
<ipython-input-38-249fd0218283>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(
<ipython-input-38-249fd0218283>:27: SettingWithCopyWarning: 
A value is trying t

In [ ]:
df.head(2)

,url,title,topic,datetime,content,subtitle
0,https://lenta.ru/news/2020/01/01/trudovaya/,В России ввели электронные трудовые книжки,russia_news,2020-01-01 00:01:00,Фото: Юрий Мартьянов /« Коммерсантъ» В России...,Фото: Юрий Мартьянов /« Коммерсантъ» В ... кн...
1,https://lenta.ru/news/2020/01/01/zavorotnuk/,Заворотнюк поблагодарила поклонников за поддержку,наука и техника,2020-01-01 00:27:02,Фото: Анатолий Ломохов / Globallookpress.com П...,Фото: Анатолий Ломохов / Globallookpress.... г...


In [ ]:
df.columns

Index(['url', 'title', 'topic', 'datetime', 'content', 'subtitle'], dtype='object')

In [ ]:
df = df[["url", "title", "subtitle", "datetime", "content", "topic"]]

In [ ]:
df.shape

(30338, 6)

In [ ]:
df.to_csv("lenta_news_big_obrab", index=False)

In [ ]:
stop_words = stopwords.words("russian")
stop_words.extend(
    [
        "что",
        "это",
        "так",
        "вот",
        "быть",
        "как",
        "в",
        "—",
        "к",
        "за",
        "из",
        "из-за",
        "на",
        "ок",
        "кстати",
        "который",
        "мочь",
        "весь",
        "еще",
        "также",
        "свой",
        "ещё",
        "самый",
        "ул",
        "комментарий",
        "английский",
        "язык",
    ]
)